In [ ]:
# import necessary libraries

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import animation

%matplotlib notebook

# I. Introduction to Cellular Automata

The purpose of this notebook is to give a quick overview of what cellular automata are, how to implement one in Python, and why they are fundamentally interesting.

A [Cellular Automaton](https://en.wikipedia.org/wiki/Cellular_automaton) is at its core, a virtual world comprised of 

(i) a <b>grid</b> of <b>cells</b>, 

(ii) a space of possible <b>states</b> which each cell can be in, 

(iii) an initial state assigned to each cell in the grid, and 

(iv) a <b>rule</b> for updating the state of each cell as time goes on.

That's it! What's really remarkable about these models is that extremely simple choices of grid, state space, and rule can lead to extremely complex emergent behavior over time, as we'll soon see.

<span style="background-color:orange"><b>Note:</b></span> All of the automata we'll be looking at, in this notebook and the others, will use a two-dimensional square grid. We'll also only consider <b>binary</b> state spaces; that is to say, each cell can only be `ON` or `OFF`. Despite the simplicity, this arrangement will still allow us to see some pretty cool emergent behavior!

## The Game of Life

Perhaps the most famous cellular automaton is the "[Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life)," invented by the mathematician John Conway in 1970.

The Game of Life is described by the following rule:

When doing an update, a cell is born (switched from `OFF` to `ON`) if it has exactly 3 live neighbors (neighbors in the state `ON`), and survives (is left in the state `ON`) if it has either 2 or 3 neighbors. Otherwise it dies (is put into the state `OFF`).

When we count how many live neighbors a cell has, we look in its [Moore neighborhood](https://en.wikipedia.org/wiki/Moore_neighborhood) - the eight cells which surround the cell we're looking at. In our implementation we'll always use periodic boundary conditions, in which we identify the right edge of the grid with the left edge, and the top edge with the bottom. That means that cells on the rightmost column are neighbors of the cells on the leftmost column, and cells on the top row are neighbors of cells on the bottom row.

Let's go ahead and code up a simple implementation of this, starting by doing the necessary imports and defining a couple constants.

In [ ]:
# define constants

GRID_SIZE = 100      # we'll use a (100 cell) x (100 cell) grid
SEED = 44            # RNG seed for reproducible random initial data

# colormap for animations
# dead cells will be black, live cells will be green
cmap = ListedColormap(["black", "lime"])

We can now define our grid of cells. We'll represent the states of the cells using `0` to represent a "dead" cell in the `OFF` state, and `1` to represent a "live" cell in the `ON` state. 

For illustrative purposes, we'll just initialize our grid by randomly picking some cells to turn `ON`. Below we implement this by turning cells `ON` with a 10% probability.

In [ ]:
# initialize grid

# seed RNG (optional, used for reproducible results)
np.random.seed(SEED)

# initialize random grid
grid = np.random.choice([0,1], (GRID_SIZE, GRID_SIZE), p=[0.9,0.1])

Now, what we need to do is actually define the Game of Life rules. Here's how we'll do it. 

The rules are defined in a 2x9 Numpy array.

The first dimension (choice of row) corresponds to which state a cell is currently in, and the second dimension (choice of column) corresponds to how many live neighbors it has. 

For example if a cell is currently `OFF` - represented by a `0` - and it has `3` live neighbors, its new state will be `rules[0, 3]`, which is `1` (`ON`). This is the process of cell birth described above in the rules. 

If on the other hand, a cell is currently `ON` - represented by a `1` - but it has `8` live neighbors, its new state will be `rules[1, 8]`, which is `0` (`OFF`). The cell dies because its neighborhood is overpopulated.

In [ ]:
# rules for Conway's Game of Life
rules = np.array([[0,0,0,1,0,0,0,0,0],[0,0,1,1,0,0,0,0,0]])

Go ahead and run the cell below to see what a typical day in the Game of Life world looks like.

In [ ]:
# animate

# initialize figure
fig = plt.figure(figsize=(6,6))
ax = fig.gca()
ax.axis('off')
im = ax.imshow(grid, cmap=cmap)


# define animate function

def animate_life(n):
    global grid
    
    # count number of live neighbors for each cell
    # the Moore neighborhood of a cell is the union of its von Neumann neighborhood...
    von_neumann_neighbors = np.roll(grid, 1, 0) + np.roll(grid, -1, 0) + \
                            np.roll(grid, 1, 1) + np.roll(grid, -1, 1)
    # ...and the four cells adjacent to it diagonally 
    diag_neighbors = np.roll(np.roll(grid, 1, 0), 1, 1) + np.roll(np.roll(grid, 1, 0), -1, 1) + \
                     np.roll(np.roll(grid, -1, 0), 1, 1) + np.roll(np.roll(grid, -1, 0), -1, 1)
    num_neighbors = von_neumann_neighbors + diag_neighbors    
    
    grid = rules[grid, num_neighbors]
    im.set_data(grid)
    return im,

anim = animation.FuncAnimation(fig, animate_life, interval=88, blit=True)

After watching for a little bit, you probably spotted some of the more common emergent entities which are known to exist within the Game of Life. Likely there were some [Blinkers](https://conwaylife.com/wiki/Blinker) and [Gliders](https://conwaylife.com/wiki/Glider), and maybe even a [Pulsar](https://conwaylife.com/wiki/Pulsar). 

All of these can come from our totally random initialization. With a more careful choice of initial conditions, even more interesting phenomena emerge. 

For example, take this relatively boring-looking choice of initial state:

![shuttle initialization](img/shuttle_init.png)

When we evolve this forward using the Game of Life rules, what results is a machine that creates two "spaceships" and continually bounces them off of one another! (This construction was found on [LifeWiki](https://conwaylife.com/wiki/Queen_bee_shuttle#Gallery))

![shuttles](img/shuttles.gif)

One can also create complex entities which move around the world on their own. Here's an example of a "[Brain](https://conwaylife.com/wiki/Brain)," a fairly large creature which seems to swim around as the grid is updated.

![brain](img/brain.gif)

These .gifs were made using the helper script `make_img.py`, which is also included in this repository.

## Conclusions

Game of Life and other cellular automata occupy a fun space. The rules which govern them are simultaneously incredibly easy to understand and implement (although scaling up to humongous world sizes presents various technical challenges), AND complex enough that they can result in the emergence of highly-recognizable creatures and events. In effect, these models provide fully-functional virtual worlds to explore, which are governed by physics that is much simpler than that in our own Universe, but still contain interesting goings-on.

In fact, the description of these cellular automata aligns nicely with the way physicists understand our world today. The most fundamental formulations of modern physics have more or less done away completely with ideas of particles and objects moving around, instead describing the world in terms of <b>[fields](https://en.wikipedia.org/wiki/Field_(physics))</b>. These physical fields, like the electromagnetic field for instance, exist everywhere, taking on a value at each point of space and time, and the current understanding is that even living creatures are ultimately the result of fluctuations in the configuration of all of these fields. 

The Game of Life world seen above is similar; all of its interesting behavior is most primitively described in terms of a simple binary field, which takes a value of either one or zero at every point. The model then acts as an extremely simplified model of a world described by the physics of fields. In the real world though, the fields found in Nature also have another interesting property - they obey the rules of <b>[quantum mechanics](https://en.wikipedia.org/wiki/Quantum_field_theory)</b>.

Ultimately the goal of this project is to introduce a <i>quantum</i> cellular automaton as a virtual example of a world which shares this property. However before we can do that we'll have to first understand another example of a classical cellular automaton. Game of Life has the property that its rules are <b>irreversible</b> - that is, many initial states of the grid can evolve to the same final state, so given a particular final state, we'd have no idea how the grid evolved up to that point. This is in contrast to the way rules work in quantum theories. In a particular sense which we will define shortly, quantum evolution is always reversible. In the next notebook therefore, we'll introduce a reversible cellular automaton called <b>Critters</b>, which will be easy to make comply with quantum mechanics.